In [2]:
import pandas as pd
import numpy as np
import re
import string
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Embedding, SpatialDropout1D, GRU
from tensorflow.keras.optimizers import SGD

df = pd.read_csv('turkish_movie_sentiment_dataset.csv', sep=',')
df.head()

,comment,film_name,point
0,\n Jean Reno denince zate...,Sevginin Gücü,"5,0"
1,\n Ekşın falan izlemek is...,Sevginin Gücü,"5,0"
2,\n Bu yapım hakkında öyle...,Sevginin Gücü,"5,0"
3,\n finali yeter... (sting...,Sevginin Gücü,"5,0"
4,\n Jean Reno..\r\nbu adam...,Sevginin Gücü,"5,0"


In [3]:
#yorumların sağındaki ve solundaki boşlukların silinmesi
comment = lambda x : x[23:-24]

df["comment"] = df["comment"].apply(comment)
df["comment"].head()

0    Jean Reno denince zaten leon filmi gelir akla ...
1    Ekşın falan izlemek istiyorsanız eğer bunu izl...
2    Bu yapım hakkında öyle çok şey yazabilirim ki ...
3    finali yeter... (sting - shape of my heart)\r\...
4    Jean Reno..\r\nbu adam kusursuz biri..\r\nve o...
Name: comment, dtype: object

In [4]:
df["point"].value_counts()

4,0    19958
5,0    15873
2,5    11433
3,5     8565
4,5     6983
3,0     5499
0,5     5150
1,0     4917
2,0     2433
1,5     1464
4,6      522
3,9      211
3,8      100
3,7       55
3,2       38
3,1       26
Name: point, dtype: int64

In [5]:
#verilen puanların ilk basamağa yuvarlanması
floatize = lambda x : float(x[0:-2])
df["point"] = df["point"].apply(floatize)
df["point"].value_counts()

4.0    27463
5.0    15873
3.0    14494
2.0    13866
1.0     6381
0.0     5150
Name: point, dtype: int64

In [6]:
#3 puanın nötr olduğu için kaldırılması, 1 ve 2 puan için 0, 4 ve 5 puan için 1 atanması
df.drop(df[df["point"] == 3].index, inplace = True)
df["point"] = df["point"].replace(1, 0)
df["point"] = df["point"].replace(2, 0)
df["point"] = df["point"].replace(4, 1)
df["point"] = df["point"].replace(5, 1)
df["point"].value_counts()

1.0    43336
0.0    25397
Name: point, dtype: int64

In [7]:
df.head()

,comment,film_name,point
0,Jean Reno denince zaten leon filmi gelir akla ...,Sevginin Gücü,1.0
1,Ekşın falan izlemek istiyorsanız eğer bunu izl...,Sevginin Gücü,1.0
2,Bu yapım hakkında öyle çok şey yazabilirim ki ...,Sevginin Gücü,1.0
3,finali yeter... (sting - shape of my heart)\r\...,Sevginin Gücü,1.0
4,Jean Reno..\r\nbu adam kusursuz biri..\r\nve o...,Sevginin Gücü,1.0


In [8]:
df.reset_index(inplace = True)
df.drop("index", axis = 1, inplace = True)
df.head()

,comment,film_name,point
0,Jean Reno denince zaten leon filmi gelir akla ...,Sevginin Gücü,1.0
1,Ekşın falan izlemek istiyorsanız eğer bunu izl...,Sevginin Gücü,1.0
2,Bu yapım hakkında öyle çok şey yazabilirim ki ...,Sevginin Gücü,1.0
3,finali yeter... (sting - shape of my heart)\r\...,Sevginin Gücü,1.0
4,Jean Reno..\r\nbu adam kusursuz biri..\r\nve o...,Sevginin Gücü,1.0


In [9]:
#tüm harflerin küçük harfe çevrilmesi
df["comment"] = df["comment"].str.lower()
df.head()

,comment,film_name,point
0,jean reno denince zaten leon filmi gelir akla ...,Sevginin Gücü,1.0
1,ekşın falan izlemek istiyorsanız eğer bunu izl...,Sevginin Gücü,1.0
2,bu yapım hakkında öyle çok şey yazabilirim ki ...,Sevginin Gücü,1.0
3,finali yeter... (sting - shape of my heart)\r\...,Sevginin Gücü,1.0
4,jean reno..\r\nbu adam kusursuz biri..\r\nve o...,Sevginin Gücü,1.0


In [10]:
#noktalama işaretlerinin kaldırılması
def remove_punctuations(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, '')
    return text

df["comment"] = df['comment'].apply(remove_punctuations)
df["comment"] = df["comment"].apply(lambda x: x.replace("\r", " "))
df["comment"] = df["comment"].apply(lambda x: x.replace("\n", " "))
df.head()

,comment,film_name,point
0,jean reno denince zaten leon filmi gelir akla ...,Sevginin Gücü,1.0
1,ekşın falan izlemek istiyorsanız eğer bunu izl...,Sevginin Gücü,1.0
2,bu yapım hakkında öyle çok şey yazabilirim ki ...,Sevginin Gücü,1.0
3,finali yeter sting shape of my heart bazıl...,Sevginin Gücü,1.0
4,jean reno bu adam kusursuz biri ve oyunculug...,Sevginin Gücü,1.0


In [11]:
#yorumlardan sayıların kaldırılması
print(df.at[4,'comment'])
df['comment'] = df['comment'].str.replace('\d+', '')
print(df.at[4,'comment'])
df.head()

jean reno  bu adam kusursuz biri  ve oyunculugu müthiş  film ise başyapıt  10


<ipython-input-11-e0eb961d9e5c>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df['comment'] = df['comment'].str.replace('\d+', '')


jean reno  bu adam kusursuz biri  ve oyunculugu müthiş  film ise başyapıt  


,comment,film_name,point
0,jean reno denince zaten leon filmi gelir akla ...,Sevginin Gücü,1.0
1,ekşın falan izlemek istiyorsanız eğer bunu izl...,Sevginin Gücü,1.0
2,bu yapım hakkında öyle çok şey yazabilirim ki ...,Sevginin Gücü,1.0
3,finali yeter sting shape of my heart bazıl...,Sevginin Gücü,1.0
4,jean reno bu adam kusursuz biri ve oyunculug...,Sevginin Gücü,1.0


In [12]:
#etkisiz kelimelerin kaldırılması
stopwords = open('turkce-stop-words.txt','r').read().split()
df['comment_without_stopwords'] = df['comment'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stopwords)]))
df.head()

,comment,film_name,point,comment_without_stopwords
0,jean reno denince zaten leon filmi gelir akla ...,Sevginin Gücü,1.0,jean reno denince leon filmi gelir akla izleme...
1,ekşın falan izlemek istiyorsanız eğer bunu izl...,Sevginin Gücü,1.0,ekşın izlemek istiyorsanız eğer izlemeyiin dos...
2,bu yapım hakkında öyle çok şey yazabilirim ki ...,Sevginin Gücü,1.0,yapım hakkında öyle çok şey yazabilirim kitap ...
3,finali yeter sting shape of my heart bazıl...,Sevginin Gücü,1.0,finali yeter sting shape of my heart bazıları ...
4,jean reno bu adam kusursuz biri ve oyunculug...,Sevginin Gücü,1.0,jean reno adam kusursuz oyunculugu müthiş film...


In [13]:
#train ve test verilerinin oluşturulması (%80 train, %20 test)
data = df["comment_without_stopwords"].values.tolist()
target = df["point"].values.tolist()

X_train, X_test, y_train, y_test = train_test_split(data,target,test_size=0.2, random_state = 42)

In [14]:
#tokenleştirme işlemi
num_words = 10000
tokenizer = Tokenizer(num_words = num_words)
tokenizer.fit_on_texts(data)
tokenizer.word_index

{'film': 1,
 'çok': 2,
 'filmi': 3,
 'iyi': 4,
 'filmin': 5,
 'için': 6,
 'güzel': 7,
 'bi': 8,
 'filmde': 9,
 'bence': 10,
 'kötü': 11,
 'değil': 12,
 'hiç': 13,
 'gerçekten': 14,
 'filme': 15,
 'ilk': 16,
 'son': 17,
 'yok': 18,
 'zaman': 19,
 'kesinlikle': 20,
 'başarılı': 21,
 'şey': 22,
 'tek': 23,
 'tavsiye': 24,
 'sadece': 25,
 'büyük': 26,
 'olmuş': 27,
 'harika': 28,
 'böyle': 29,
 'fazla': 30,
 'olduğunu': 31,
 'cok': 32,
 'olduğu': 33,
 'nasıl': 34,
 'izledim': 35,
 'senaryo': 36,
 'konu': 37,
 'aynı': 38,
 'sinema': 39,
 'farklı': 40,
 'özellikle': 41,
 'şekilde': 42,
 'önce': 43,
 'komedi': 44,
 'izlediğim': 45,
 'mükemmel': 46,
 'filmleri': 47,
 'gereken': 48,
 'yer': 49,
 'aksiyon': 50,
 'göre': 51,
 'korku': 52,
 'türk': 53,
 'oyuncu': 54,
 'oldukça': 55,
 'filmden': 56,
 'yönetmen': 57,
 'rağmen': 58,
 'filmlerden': 59,
 'bazı': 60,
 'izleyin': 61,
 'uzun': 62,
 'tüm': 63,
 'olması': 64,
 'başka': 65,
 'konusu': 66,
 'filmler': 67,
 'içinde': 68,
 'çünkü': 69,
 'filmdi

In [15]:
#train ve test verilerini tokenleştirme
X_train_tokens = tokenizer.texts_to_sequences(X_train)
X_test_tokens = tokenizer.texts_to_sequences(X_test)

print(X_train[20])
print(X_train_tokens[20])
df.head()

moviemaxte vermeye başladıklarından seyrettim seyretmeye devam edicemmtürünün izlenesi örneklerindendirbeğenmeyenlere çok şaşırıyorumbi korku filmi şaşırtabilir
[1768, 924, 2562, 95, 1559, 2, 52, 3]


,comment,film_name,point,comment_without_stopwords
0,jean reno denince zaten leon filmi gelir akla ...,Sevginin Gücü,1.0,jean reno denince leon filmi gelir akla izleme...
1,ekşın falan izlemek istiyorsanız eğer bunu izl...,Sevginin Gücü,1.0,ekşın izlemek istiyorsanız eğer izlemeyiin dos...
2,bu yapım hakkında öyle çok şey yazabilirim ki ...,Sevginin Gücü,1.0,yapım hakkında öyle çok şey yazabilirim kitap ...
3,finali yeter sting shape of my heart bazıl...,Sevginin Gücü,1.0,finali yeter sting shape of my heart bazıları ...
4,jean reno bu adam kusursuz biri ve oyunculug...,Sevginin Gücü,1.0,jean reno adam kusursuz oyunculugu müthiş film...


In [16]:
#padding işlemi
maxlen = 150
X_train_pad = pad_sequences(X_train_tokens, maxlen = maxlen) 
X_test_pad = pad_sequences(X_test_tokens, maxlen = maxlen)

print(X_train_pad.shape)
print(X_test_pad.shape)

(54986, 150)
(13747, 150)


In [24]:
embedding_out = 50
lstm_out = 196
model = Sequential()
model.add(Embedding(input_dim = num_words, output_dim = embedding_out, input_length = maxlen))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation = "sigmoid"))
model.compile(optimizer = 'adam', loss = "binary_crossentropy", metrics = ["accuracy"])
model.summary()
X_train_pad = np.array(X_train_pad)
y_train = np.array(y_train)

model.fit(X_train_pad, y_train, epochs = 5, batch_size = 256)

model.save("model1.h5")


Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 150, 50)           500000    
_________________________________________________________________
spatial_dropout1d (SpatialDr (None, 150, 50)           0         
_________________________________________________________________
lstm (LSTM)                  (None, 196)               193648    
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 197       
Total params: 693,845
Trainable params: 693,845
Non-trainable params: 0
_________________________________________________________________
Epoch 1/5
215/215 [==============================] - 200s 919ms/step - loss: 0.5190 - accuracy: 0.7439
Epoch 2/5
215/215 [==============================] - 198s 921ms/step - loss: 0.4123 - accuracy: 0.8211
Epoch 3/5
215/215 [=====================

In [25]:
embedding_out = 50
model2 = Sequential()
model2.add(Embedding(input_dim = num_words, output_dim = embedding_out, input_length = maxlen))
model2.add(GRU(units = 32, return_sequences = True))
model2.add(GRU(units = 16, return_sequences = True))
model2.add(GRU(units = 8))
model2.add(Dense(1, activation = "sigmoid"))
model2.compile(optimizer = 'adam', loss = "binary_crossentropy", metrics = ["accuracy"])
model2.summary()
X_train_pad = np.array(X_train_pad)
y_train = np.array(y_train)

model2.fit(X_train_pad, y_train, epochs = 5, batch_size = 256)
model2.save("model2.h5")

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 150, 50)           500000    
_________________________________________________________________
gru_11 (GRU)                 (None, 150, 32)           8064      
_________________________________________________________________
gru_12 (GRU)                 (None, 150, 16)           2400      
_________________________________________________________________
gru_13 (GRU)                 (None, 8)                 624       
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 9         
Total params: 511,097
Trainable params: 511,097
Non-trainable params: 0
_________________________________________________________________
Epoch 1/5
215/215 [==============================] - 9s 23ms/step - loss: 0.5088 - accuracy: 0.7496
Epoch 2/5
215/

In [26]:
embedding_out = 50
model3 = Sequential()
model3.add(Embedding(input_dim = num_words, output_dim = embedding_out, input_length = maxlen))
model3.add(GRU(units = 32, return_sequences = True))
model3.add(GRU(units = 16, return_sequences = True))
model3.add(GRU(units = 8, return_sequences = True))
model3.add(GRU(units = 4))
model3.add(Dense(1, activation = "sigmoid"))

opt=SGD(learning_rate=0.1,momentum=0.9)

model3.compile(optimizer = opt, loss = "binary_crossentropy", metrics = ["accuracy"])
model3.summary()
X_train_pad = np.array(X_train_pad)
y_train = np.array(y_train)

model3.fit(X_train_pad, y_train, epochs = 5, batch_size = 256)
model3.save("model3.h5")

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 150, 50)           500000    
_________________________________________________________________
gru_14 (GRU)                 (None, 150, 32)           8064      
_________________________________________________________________
gru_15 (GRU)                 (None, 150, 16)           2400      
_________________________________________________________________
gru_16 (GRU)                 (None, 150, 8)            624       
_________________________________________________________________
gru_17 (GRU)                 (None, 4)                 168       
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 5         
Total params: 511,261
Trainable params: 511,261
Non-trainable params: 0
________________________________________________

In [27]:
#tek bir yorum için duygu tahmini yapılması
def comment_pred(comment, model):
    comment = [comment]
    comment = tokenizer.texts_to_sequences(comment)
    comment = pad_sequences(comment, maxlen=maxlen, dtype='int32', value=0)

    sentiment = model.predict(comment,batch_size=1,verbose = 2)[0]

    if(sentiment < 0.5):
        print("negative")
    elif (sentiment >= 0.5):
        print("positive")

    print(sentiment)


comment = 'çok güzel bir film'
comment_pred(comment, model)
comment_pred(comment, model2)
comment_pred(comment, model3)

1/1 - 0s
positive
[0.79165864]
1/1 - 1s
positive
[0.8537489]
1/1 - 1s
positive
[0.6655347]


In [29]:
#oluşturulan modelin test verisindeki yorumlar için yapılan yanlış tahmin sayıları
def uncorrect_preds(model):
    y_pred = model.predict(X_test_pad, verbose = 1)
    y_pred = y_pred.T[0]
    array_pred = np.array([1.0 if p > 0.5 else 0.0 for p in y_pred])
    array_true = np.array(y_test)
    incorrect = np.where(array_pred != array_true)
    incorrect = incorrect[0]
    print(len(incorrect))


uncorrect_preds(model)
uncorrect_preds(model2)
uncorrect_preds(model3)

430/430 [==============================] - 46s 106ms/step
2877
430/430 [==============================] - 2s 5ms/step
2902
430/430 [==============================] - 3s 6ms/step
3032
